<a href="https://colab.research.google.com/github/SaimManzoor49/tensorflow/blob/main/LinearRegressionModel_Titanic_Predection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import sklearn as sl
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import pandas as pd
from IPython.display import clear_output
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc
import tensorflow as tf


In [21]:
# get train data and evaluation data
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv')
dfeval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv')
y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived')

# seperate Categorical(true,false)||(male,female)||(any thing like that) and numerical values(1,2,5,6,7.787,454354)
CATEGORICAL_COLUMNS = ['sex','n_siblings_spouses','parch','class','deck','embark_town','alone']
NUMERIC_COLUMNS = ['age','fare']

feature_columns = [] # formating or preparing data for training

for feature_name in CATEGORICAL_COLUMNS:
  vocabulary = dftrain[feature_name].unique() # gives all unique values in the categorical_column of name feature_name in train ds
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name,vocabulary)) # addind it to FC

for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name,dtype=tf.float32)) # gives all unique values in the numeric_column of name feature_name in train ds

# traning modal
# input function to manage traning data into batches and epoch , encodes data in tf.data.Dataset object required for training

def make_input_function(data_df,label_df,num_epochs=100,shuffle=True,batch_size=32): # returns the requires input function for traning
  def input_function():
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df),label_df)) # creates tf.data.Datasets object with data and its lables
    if shuffle:
      ds = ds.shuffle(1000)
    ds = ds.batch(batch_size).repeat(num_epochs) # split in batch size and repeat for num of epochs
    return ds
  return input_function


train_input_fn = make_input_function(dftrain,y_train) # for training
eval_input_fn = make_input_function(dfeval,y_eval,num_epochs=1,shuffle=False) # for eval

# creating the modal
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)

# traning modal
linear_est.train(train_input_fn)

# evaluating the modal
result = linear_est.evaluate(eval_input_fn)

clear_output()
print(result['accuracy'])
print(result)


0.77272725
{'accuracy': 0.77272725, 'accuracy_baseline': 0.625, 'auc': 0.837588, 'auc_precision_recall': 0.7879456, 'average_loss': 0.47431797, 'label/mean': 0.375, 'loss': 0.45775047, 'precision': 0.6857143, 'prediction/mean': 0.41460276, 'recall': 0.72727275, 'global_step': 2000}


In [30]:
pridection = list(linear_est.predict(eval_input_fn))
print(dfeval.loc[10])
print(pridection[10]['probabilities'][1])

sex                       female
age                         28.0
n_siblings_spouses             0
parch                          0
fare                        7.75
class                      Third
deck                     unknown
embark_town           Queenstown
alone                          y
Name: 10, dtype: object
0.7193596
